## Task 5 - Artyom Novikov J41332c 

### Make all the necessary imports

In [1]:
import re 
import math
from textblob import TextBlob as tb
from nltk.corpus import wordnet as wn
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
from collections import Counter

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


### First step. Download the text and perform all the necessary preprocessing on the text.

In [2]:
alice_story = open("alice.txt", 'rb').readlines()
alice_story = list(map(lambda s:s.decode("utf8"), alice_story))

### Lowercase text


In [3]:
alice_story = "\n".join(alice_story).lower()

### Take only word from the text

In [4]:
words_only = re.sub("\W", " ", alice_story)[1547:] 

### Creating a list of chapters and function for taking the text from the chapter

In [5]:
chapters = ['i', 'ii', 'iii', 'iv', 'v', 'vi', 'vii', 'viii', 'ix', 'x', 'xi', 'xii', 'the end']

In [6]:
def get_text(chapter = 1):
    if chapter != 11:
        reg = "chapter %s .{100,} chapter %s "%(chapters[chapter], chapters[chapter+1])
    else:
        reg = "chapter %s .{100,} %s "%(chapters[chapter], chapters[chapter+1])
    return re.search(reg, words_only)[0]

In [17]:
print(get_text(chapter = 0))
print(get_text(chapter = 7))

chapter i    down the rabbit hole         alice was beginning to get very tired of sitting by her sister on the   bank  and of having nothing to do  once or twice she had peeped into   the book her sister was reading  but it had no pictures or   conversations in it   and what is the use of a book   thought alice    without pictures or conversations        so she was considering in her own mind  as well as she could  for the   hot day made her feel very sleepy and stupid   whether the pleasure of   making a daisy chain would be worth the trouble of getting up and   picking the daisies  when suddenly a white rabbit with pink eyes ran   close by her       there was nothing so _very_ remarkable in that  nor did alice think it   so _very_ much out of the way to hear the rabbit say to itself   oh   dear  oh dear  i shall be late    when she thought it over afterwards    it occurred to her that she ought to have wondered at this  but at the   time it all seemed quite natural   but when the ra

### Top 10 most important words from each chapter in the text (not "Alice")

In [8]:
def tf(word, blob):
    return blob.split().count(word) / len(blob.split())
    
def idf(word, bloblist):
    return math.log(len(bloblist) / (1 + n_containing(word, bloblist)))

def n_containing(word, bloblist):
    return sum(1 for blob in bloblist if word in blob.split())

def tfidf(word, blob, bloblist):
    return tf(word, blob) * idf(word, bloblist)

In [9]:
bloblist = [get_text(i) for i in range(len(chapters)-1)]
for i, blob in enumerate(bloblist):
    print("Chapter %s "%chapters[i])
    scores = {word: tfidf(word, blob, bloblist) for word in blob.split()}
    sorted_words = sorted(scores.items(), key=lambda x: x[1], reverse=True)
    for word, score in sorted_words[:10]:
        print("\tword: {}, tf-idf score: {}".format(word, round(score, 5)))

Chapter i 
	word: bats, tf-idf score: 0.00326
	word: key, tf-idf score: 0.003
	word: hole, tf-idf score: 0.00252
	word: bottle, tf-idf score: 0.00252
	word: dark, tf-idf score: 0.00245
	word: candle, tf-idf score: 0.00245
	word: cake, tf-idf score: 0.00245
	word: eat, tf-idf score: 0.00221
	word: fell, tf-idf score: 0.002
	word: cats, tf-idf score: 0.002
Chapter ii 
	word: mouse, tf-idf score: 0.00492
	word: swam, tf-idf score: 0.00409
	word: pool, tf-idf score: 0.00401
	word: cats, tf-idf score: 0.00351
	word: mabel, tf-idf score: 0.00327
	word: gloves, tf-idf score: 0.00253
	word: fan, tf-idf score: 0.00253
	word: dogs, tf-idf score: 0.00245
	word: tears, tf-idf score: 0.0019
	word: kid, tf-idf score: 0.0019
Chapter iii 
	word: dodo, tf-idf score: 0.00954
	word: mouse, tf-idf score: 0.00618
	word: race, tf-idf score: 0.00616
	word: prizes, tf-idf score: 0.00514
	word: lory, tf-idf score: 0.00477
	word: thimble, tf-idf score: 0.00411
	word: dry, tf-idf score: 0.00378
	word: caucus, tf

### *According to the results chapters could be named by the most important word, or combination of most important words


### To find the Top 10 most used verbs in sentences with Alice, creating a fuction to check if word is a verb

In [10]:
def is_verb(tag):
    """
    Convert POS tag from Penn tagset to WordNet tagset.
    :param tag: a tag from Penn tagset
    :return: a tag from WordNet tagset or None if no corresponding tag could be found
    """
    if tag in ['VB', 'VBD', 'VBG', 'VBN', 'VBP', 'VBZ']:
        return True
    return False 

In [11]:
clear_sentences = []
for sentence in alice_story.split("."):
    clear_sentence = re.sub("\W", " ", sentence)
    if "alice" in clear_sentence:
        clear_sentences.append(clear_sentence)

In [12]:
most_common_verbs = []
for sentence in clear_sentences:
    words = nltk.pos_tag(nltk.word_tokenize(sentence)) 
    for word in words:
        if is_verb(word[1]) and len(word[0])>2:
            most_common_verbs.append(word[0])

In [13]:
Counter(most_common_verbs).most_common(10)

[('said', 209),
 ('was', 184),
 ('had', 96),
 ('thought', 45),
 ('went', 43),
 ('were', 41),
 ('know', 41),
 ('see', 36),
 ('say', 34),
 ('have', 34)]

### What does Alice do most often? Answer: speaks 